 # **Classificação de imagens cachorros e gatos**

# Problema

Este é um projeto da disciplina de Processamento de Imagens, da UTFPR-CP, cujo o objetivo é criar uma arquitetura de Convolutional Neural Network (CNN) para classificação de imagens de um problema qualquer. Posto isso, escolheu-se classificar cachorros e gatos, utilizando o conjunto de dados disponibilizado no Kaggle, podendo encontra-lo [aqui](https://www.kaggle.com/competitions/dogs-vs-cats).

Ao longo do notebook será realizado

* Técnicas de processamento de imagens;
* Criação da CNN, utilizando o TensorFlow/Keras;

# Importando bibliotecas

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.optimizers import RMSprop
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Carregando os dados do dataset


In [ ]:
work_path = './cats_and_dogs_filtered'
os.mkdir(work_path)

In [ ]:
#Extraindo zip
local_zip = '../input/dogs-vs-cats/test1.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall(work_path)

local_zip = '../input/dogs-vs-cats/train.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall(work_path)

zip_ref.close()

In [ ]:
#Armazendo em variável os caminhos de treino e test1
treino_path = os.path.join(work_path, 'train')
teste_path = os.path.join(work_path, 'test1')

In [ ]:
#Dados do caminho de treino
treino_df = pd.DataFrame({'image_name':os.listdir(treino_path)})
treino_df['label'] =treino_df['image_name'].apply(lambda x: x.split('.')[0])
treino_df

In [ ]:
#Dados do caminho de test
teste_df = pd.DataFrame({'image_name':os.listdir(teste_path)})
teste_df['label'] =teste_df['image_name'].apply(lambda x: x.split('.')[0])
teste_df

In [ ]:
#Dados do treino com label dog
cachorro_path_treino = os.path.join(treino_path, 'dog')
os.mkdir(cachorro_path_treino)
cachorro_df_treino = treino_df[treino_df.label=='dog']
for n in tqdm(cachorro_df_treino.image_name):
    os.rename((os.path.join(treino_path, n)), (os.path.join(cachorro_path_treino, n)))

In [ ]:
#Dados do treino com label cat
gato_path_treino = os.path.join(treino_path, 'cat')
os.mkdir(gato_path_treino)
gato_df_treino = treino_df[treino_df.label=='cat']
for n in tqdm(gato_df_treino.image_name):
    os.rename((os.path.join(treino_path, n)), (os.path.join(gato_path_treino, n)))

In [ ]:
# Mostrando conteúdos dos subdiretorios do dataset, o principal, o de treino e o de validação
base_dir = './cats_and_dogs_filtered'

print(' Conteúdo do diretório base')
print(os.listdir(base_dir))

print('\n Conteúdo do diretório de treino')
train_path = f'{base_dir}/train'
print(os.listdir(treino_path))

print('\n Conteúdo do diretório de validação')
print(os.listdir(teste_path)[:10])

In [ ]:
# Armazenando o diretório em uma variável
treino_dir = os.path.join(base_dir,'train')
validacao_dir = os.path.join(base_dir,'test1')

treino_gatos_dir = os.path.join(treino_dir,'cat')
treino_cachorros_dir = os.path.join(treino_dir,'dog')

In [ ]:
# Mostrando nome dos arquivos
treino_gatos_nomes = os.listdir(treino_gatos_dir)
treino_cachorros_nomes = os.listdir(treino_cachorros_dir)

print(treino_gatos_nomes[:10])
print(treino_cachorros_nomes[:10])

In [ ]:
# Quantidade de cachorros e gatos nos conjuntos de treino e validação
print(f'Quantidade de gatos no conjunto de treino = {len(treino_gatos_nomes)}')
print(f'Quantidade de cachorros no conjunto de treino = {len(treino_cachorros_nomes)}')
print(f'Quantidade de cachorros e gatos no conjunto de validação = {len(os.listdir(validacao_dir))}')

In [ ]:
# Plotando algumas imagens de cachorros e gatos
%matplotlib inline

nlinhas = 2
ncolunas = 4
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(nlinhas*4,ncolunas*1)

proxima_foto_gato = [os.path.join(treino_gatos_dir,fname) for fname in treino_gatos_nomes[pic_index:pic_index+4]]
proxima_foto_cachorro = [os.path.join(treino_cachorros_dir,fname) for fname in treino_cachorros_nomes[pic_index:pic_index+4]]

for i ,img_path in enumerate(proxima_foto_gato+proxima_foto_cachorro):
    sp = plt.subplot(nlinhas,ncolunas,i+1)
    sp.axis('off')
    
    img = mpimg.imread(img_path)
    plt.imshow(img)
plt.show()    
    

# Construindo a arquitetura

In [ ]:
#Criação do modelo
def create_model():

  model = tf.keras.models.Sequential([ 
      
      tf.keras.layers.Conv2D(16,(3,3), activation = 'relu', input_shape=(150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),

      tf.keras.layers.Conv2D(32,(3,3), activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),

      tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2,2),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation = 'relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  #Compilando o modelo
  model.compile(optimizer=RMSprop(lr=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy']) 
    
  return model

In [ ]:
model = create_model()
model.summary()

# ImageDataGenerator

In [ ]:
#Treinando DataAugmentation e Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

treino_datagen = ImageDataGenerator(rescale=1./255,
                 rotation_range=40,
                 width_shift_range=0.2,
                 height_shift_range=0.2,
                 shear_range=0.2,
                 zoom_range=0.2,
                 horizontal_flip=True,
                 fill_mode='nearest',
                 validation_split=0.2
) # definindo divisão do conjunto de validação

treino_generator = treino_datagen.flow_from_directory(
                   treino_dir,
                   target_size=(150,150),
                   batch_size=50,
                   class_mode='binary',
                   subset='training'
) # definindo dados para treino

validacao_generator = treino_datagen.flow_from_directory(
                      treino_dir, 
                      target_size=(150, 150),
                      batch_size=50,
                      class_mode='binary',
                      subset='validation'
) # definindo dados para validação

In [ ]:
#Função callback para interromper o treinamento quando val_accuracy maior ou igual a 90%
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('val_accuracy')>=0.90):
            self.model.stop_training = True
            
callback = mycallback()

# Treinando modelo

In [ ]:
#Treinando o modelo
history = model.fit(
    treino_generator,
    steps_per_epoch = treino_generator.samples    //50,#batch_size, 
    epochs = 20,
    verbose=1,
    validation_data = validacao_generator,
    validation_steps = validacao_generator.samples    //50,#batch_size,
    callbacks=[callback]
)

# Acurácia do treino e validação

In [ ]:
#Plotando grafico de acur
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, 'b', label='Acurácia do treino')
plt.plot(val_acc, 'r', label='Acurácia da validação')
plt.legend(loc='lower right')
plt.ylabel('Acurácia')
plt.ylim([min(plt.ylim()),1])
plt.title('Acurácia do treino e da validação')

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(2, 1, 2)
plt.plot(loss, 'b', label='Loss do treino')
plt.plot(val_loss, 'r', label='Loss da validação')
plt.legend(loc='upper right')
plt.ylabel('Entropia cruzada')
plt.ylim([0,1.0])
plt.title('Loss do treino e validação')
plt.xlabel('epoch')
plt.show()